In [0]:
pip install unidecode

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import re
import os
from unidecode import unidecode
from datetime import datetime
from difflib import SequenceMatcher
import numpy as np
import sys

def limpiar_texto(texto):
    if pd.isnull(texto):
        return ""
    
    texto = texto.lower()
    texto = unidecode(texto)
    texto = texto.replace(",", " ")
    texto = re.sub(r"[^a-zA-Z0-9\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

def limpiar_topico(texto):
    if pd.isnull(texto):
        return ""
    
    texto = unidecode(texto)
    texto = re.sub(r"[^a-zA-Z0-9_\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

def es_palabra_comun(palabra):
    palabras_comunes = {
        "bien", "esta", "todo", "para", "por", "con", "los", "las", "que", "una", "uno", 
        "mas", "muy", "hay", "solo", "como", "pero", "cuando", "hora", "hacer", "haces",
        "dan", "dar", "deben", "estan", "esta", "en", "el", "la", "de", "del", "y", "o",
        "me", "mi", "se", "su", "lo", "tan", "te", "tu", "si", "no", "mis", "sus", "fue",
        "son", "ser", "sin", "fue", "era", "eso", "esto", "esa", "ese", "este", "estos",
        "aqui", "ahi", "alla", "cada", "vez", "otro", "otra", "otros", "otras", "hasta",
        "desde", "sobre", "bajo", "entre", "hacia", "contra", "segun", "durante", "mediante",
        "al", "ya", "mas", "menos", "poco", "mucho", "dentro", "fuera", "antes", "despues",
        "entonces", "luego", "ahora", "siempre", "nunca", "a", "ha"
    }
    return palabra.lower().strip() in palabras_comunes

def normalizar_palabra(palabra):
    palabra = palabra.lower().strip()
    
    sufijos = ['s', 'es', 'ar', 'er', 'ir', 'ando', 'endo', 'cion', 'sion']
    for sufijo in sufijos:
        if len(palabra) > len(sufijo) + 3 and palabra.endswith(sufijo):
            palabra = palabra[:-len(sufijo)]
            break
    
    return palabra

def similitud_levenshtein(s1, s2):
    if s1 == s2:
        return 1.0
        
    len_s1, len_s2 = len(s1), len(s2)
    
    if len_s1 == 0: 
        return 0.0
    if len_s2 == 0:
        return 0.0
        
    matriz = [[0 for _ in range(len_s2 + 1)] for _ in range(len_s1 + 1)]
    
    for i in range(len_s1 + 1):
        matriz[i][0] = i
    for j in range(len_s2 + 1):
        matriz[0][j] = j
        
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            if s1[i-1] == s2[j-1]:
                costo = 0
            else:
                costo = 1
                
            matriz[i][j] = min(
                matriz[i-1][j] + 1,
                matriz[i][j-1] + 1,
                matriz[i-1][j-1] + costo
            )
    
    distancia = matriz[len_s1][len_s2]
    
    max_len = max(len_s1, len_s2)
    if max_len == 0:
        return 0.0
    
    similitud = 1.0 - (distancia / max_len)
    
    return similitud

def similitud_palabras_mejorada(palabra1, palabra2):
    pal1 = palabra1.lower().strip()
    pal2 = palabra2.lower().strip()
    
    if pal1 == pal2:
        return 1.0
    
    if len(pal1) <= 3 or len(pal2) <= 3:
        return 1.0 if pal1 == pal2 else 0.0
    
    if es_palabra_comun(pal1) or es_palabra_comun(pal2):
        return 1.0 if pal1 == pal2 else 0.0
    
    norm_pal1 = normalizar_palabra(pal1)
    norm_pal2 = normalizar_palabra(pal2)
    
    if norm_pal1 == norm_pal2:
        return 0.95
    
    similitud = similitud_levenshtein(pal1, pal2)
    
    longitud_promedio = (len(pal1) + len(pal2)) / 2
    umbral_base = 0.7
    
    if longitud_promedio > 8:
        umbral_adaptado = umbral_base - 0.05
    elif longitud_promedio < 5:
        umbral_adaptado = umbral_base + 0.15
    else:
        umbral_adaptado = umbral_base
    
    return similitud if similitud >= umbral_adaptado else 0.0

def buscar_patron_exacto(comentario, frase_exacta):
    comentario_norm = ' '.join(comentario.split())
    frase_norm = ' '.join(frase_exacta.split())
    
    return comentario_norm == frase_norm

def cargar_n1(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"El archivo {path} no existe.")
    
    dic_n1 = {}
    categoria_actual = None
    
    with open(path, "r", encoding="utf-8") as f:
        for linea_num, linea in enumerate(f, 1):
            linea = linea.strip()
            if not linea:
                continue
            
            if linea.startswith("#") and not linea.startswith("##"):
                categoria_actual = limpiar_texto(linea.replace("#", "").strip())
                if categoria_actual in dic_n1:
                    print(f"Advertencia en línea {linea_num}: Categoría N1 '{categoria_actual}' ya existe. Sobrescribiendo.")
                dic_n1[categoria_actual] = {"patron": [], "exacto": []}
            else:
                es_exacto = linea.startswith("- ")
                if es_exacto:
                    frase = limpiar_texto(linea[2:])
                else:
                    frase = limpiar_texto(linea)
                
                frase = ' '.join(frase.split())
                if categoria_actual and frase:
                    if es_exacto:
                        dic_n1[categoria_actual]["exacto"].append(frase)
                    else:
                        dic_n1[categoria_actual]["patron"].append(frase)
                elif frase:
                    print(f"Advertencia en línea {linea_num}: Frase clave '{frase}' encontrada sin una categoría N1 previa.")
    
    return dic_n1

# Función para cargar el archivo de exclusiones
def cargar_exclusiones(path):
    if not os.path.exists(path):
        print(f"Advertencia: El archivo de exclusiones {path} no existe.")
        return {}
    
    dic_exclusiones = {}
    categoria_actual = None
    
    with open(path, "r", encoding="utf-8") as f:
        for linea_num, linea in enumerate(f, 1):
            linea = linea.strip()
            if not linea:
                continue
            
            if linea.startswith("#"):
                categoria_actual = limpiar_texto(linea.replace("#", "").strip())
                if categoria_actual in dic_exclusiones:
                    print(f"Advertencia en línea {linea_num}: Categoría '{categoria_actual}' ya existe en exclusiones. Continuando con la existente.")
                else:
                    dic_exclusiones[categoria_actual] = []
            elif categoria_actual:
                patron = limpiar_texto(linea)
                if patron:
                    dic_exclusiones[categoria_actual].append(patron)
            else:
                print(f"Advertencia en línea {linea_num}: Patrón de exclusión '{linea}' encontrado sin una categoría previa.")
    
    return dic_exclusiones

def cargar_n2(path, dic_n1):
    if not os.path.exists(path):
        raise FileNotFoundError(f"El archivo {path} no existe.")
    
    dic_n2 = {}
    categoria_n1_actual = None
    subcategoria_n2_actual = None
    
    with open(path, "r", encoding="utf-8") as f:
        for linea_num, linea in enumerate(f, 1):
            linea = linea.strip()

            if not linea:
                continue

            if linea.startswith("#") and not linea.startswith("##"):
                categoria_n1 = limpiar_texto(linea.replace("#", "").strip())
                if categoria_n1 not in dic_n1:
                    print(f"Advertencia en línea {linea_num}: Categoría N1 '{categoria_n1}' en n2.txt no existe en n1.txt.")
                categoria_n1_actual = categoria_n1
                if categoria_n1_actual not in dic_n2:
                    dic_n2[categoria_n1_actual] = {}
                subcategoria_n2_actual = None

            elif linea.startswith("##"):
                if categoria_n1_actual is None:
                    print(f"Advertencia en línea {linea_num}: Subcategoría N2 encontrada antes de una categoría N1.")
                    continue
                
                subcategoria_n2 = limpiar_texto(linea.replace("##", "").strip())
                if subcategoria_n2 in dic_n2[categoria_n1_actual]:
                    print(f"Advertencia en línea {linea_num}: Subcategoría N2 '{subcategoria_n2}' ya existe bajo la categoría N1 '{categoria_n1_actual}'. Se ignorará la duplicación.")
                    continue
                dic_n2[categoria_n1_actual][subcategoria_n2] = {"patron": [], "exacto": []}
                subcategoria_n2_actual = subcategoria_n2
            else:
                if categoria_n1_actual is None or subcategoria_n2_actual is None:
                    print(f"Advertencia en línea {linea_num}: Frase clave encontrada fuera de contexto en la línea: '{linea}'")
                    continue
                
                es_exacto = linea.startswith("- ")
                if es_exacto:
                    frase = limpiar_texto(linea[2:])
                else:
                    frase = limpiar_texto(linea)
                    
                frase = ' '.join(frase.split())
                
                if frase:
                    if es_exacto:
                        dic_n2[categoria_n1_actual][subcategoria_n2_actual]["exacto"].append(frase)
                    else:
                        dic_n2[categoria_n1_actual][subcategoria_n2_actual]["patron"].append(frase)
    
    return dic_n2

def cargar_n3(path, dic_n1, dic_n2):
    if not os.path.exists(path):
        raise FileNotFoundError(f"El archivo {path} no existe.")
    
    dic_n3 = {}
    categoria_n1_actual = None
    subcategoria_n2_actual = None
    subcategoria_n3_actual = None
    
    with open(path, "r", encoding="utf-8") as f:
        for linea_num, linea in enumerate(f, 1):
            linea = linea.strip()

            if not linea:
                continue

            if linea.startswith("#") and not linea.startswith("##") and not linea.startswith("###"):
                categoria_n1 = limpiar_texto(linea.replace("#", "").strip())
                if categoria_n1 not in dic_n1:
                    print(f"Advertencia en línea {linea_num}: Categoría N1 '{categoria_n1}' en n3.txt no existe en n1.txt.")
                categoria_n1_actual = categoria_n1
                if categoria_n1_actual not in dic_n3:
                    dic_n3[categoria_n1_actual] = {}
                subcategoria_n2_actual = None
                subcategoria_n3_actual = None

            elif linea.startswith("##") and not linea.startswith("###"):
                if categoria_n1_actual is None:
                    print(f"Advertencia en línea {linea_num}: Subcategoría N2 encontrada antes de una categoría N1.")
                    continue
                
                subcategoria_n2 = limpiar_texto(linea.replace("##", "").strip())
                if categoria_n1_actual not in dic_n2 or subcategoria_n2 not in dic_n2[categoria_n1_actual]:
                    print(f"Advertencia en línea {linea_num}: Subcategoría N2 '{subcategoria_n2}' en n3.txt no existe en n2.txt para la categoría N1 '{categoria_n1_actual}'.")
                
                if subcategoria_n2 not in dic_n3.get(categoria_n1_actual, {}):
                    dic_n3[categoria_n1_actual][subcategoria_n2] = {}
                subcategoria_n2_actual = subcategoria_n2
                subcategoria_n3_actual = None
                
            elif linea.startswith("###"):
                if categoria_n1_actual is None or subcategoria_n2_actual is None:
                    print(f"Advertencia en línea {linea_num}: Subcategoría N3 encontrada antes de categorías N1 o N2.")
                    continue
                
                subcategoria_n3 = limpiar_texto(linea.replace("###", "").strip())
                if subcategoria_n3 in dic_n3.get(categoria_n1_actual, {}).get(subcategoria_n2_actual, {}):
                    print(f"Advertencia en línea {linea_num}: Subcategoría N3 '{subcategoria_n3}' ya existe bajo la categoría N1 '{categoria_n1_actual}' y N2 '{subcategoria_n2_actual}'. Se ignorará la duplicación.")
                    continue
                
                dic_n3[categoria_n1_actual][subcategoria_n2_actual][subcategoria_n3] = {"patron": [], "exacto": []}
                subcategoria_n3_actual = subcategoria_n3
                
            else:
                if categoria_n1_actual is None or subcategoria_n2_actual is None or subcategoria_n3_actual is None:
                    print(f"Advertencia en línea {linea_num}: Frase clave encontrada fuera de contexto en la línea: '{linea}'")
                    continue
                
                es_exacto = linea.startswith("- ")
                if es_exacto:
                    frase = limpiar_texto(linea[2:])
                else:
                    frase = limpiar_texto(linea)
                    
                frase = ' '.join(frase.split())
                
                if frase:
                    if es_exacto:
                        dic_n3[categoria_n1_actual][subcategoria_n2_actual][subcategoria_n3_actual]["exacto"].append(frase)
                    else:
                        dic_n3[categoria_n1_actual][subcategoria_n2_actual][subcategoria_n3_actual]["patron"].append(frase)
    
    return dic_n3

def buscar_patron_avanzado(comentario, patron, max_gap=5):
    palabras_patron = patron.split()
    palabras_comentario = comentario.split()
    
    if len(palabras_patron) <= 2:
        palabras_significativas = [p for p in palabras_patron if not es_palabra_comun(p)]
        
        if not palabras_significativas and len(palabras_patron) == 1:
            return palabras_patron[0] in palabras_comentario
            
        if len(palabras_significativas) > 0:
            umbral_local = 0.85
            encontrados = 0
            
            for palabra_patron in palabras_significativas:
                for palabra_comentario in palabras_comentario:
                    similitud = similitud_palabras_mejorada(palabra_comentario, palabra_patron)
                    if similitud >= umbral_local:
                        encontrados += 1
                        break
            
            return encontrados == len(palabras_significativas)
    
    indices_por_palabra = []
    
    for palabra_patron in palabras_patron:
        indices = []
        for i, palabra_comentario in enumerate(palabras_comentario):
            if not es_palabra_comun(palabra_patron) and len(palabra_patron) > 3:
                similitud = similitud_palabras_mejorada(palabra_comentario, palabra_patron)
                if similitud >= 0.85:
                    indices.append(i)
            elif palabra_comentario == palabra_patron:
                indices.append(i)
        
        if not indices:
            return False
            
        indices_por_palabra.append(indices)
    
    return verificar_secuencia_valida(indices_por_palabra, max_gap)

def verificar_secuencia_valida(indices_por_palabra, max_gap):
    if len(indices_por_palabra) == 1:
        return True
        
    def buscar_secuencia(posicion_actual, indice_actual, visitados=None):
        if visitados is None:
            visitados = set()
            
        visitados.add((posicion_actual, indice_actual))
        
        if indice_actual == len(indices_por_palabra) - 1:
            return True
            
        for siguiente_indice in range(indice_actual + 1, len(indices_por_palabra)):
            if (posicion_actual, siguiente_indice) in visitados:
                continue
                
            for siguiente_pos in indices_por_palabra[siguiente_indice]:
                distancia = siguiente_pos - posicion_actual
                if 0 < distancia <= max_gap + 1:
                    if buscar_secuencia(siguiente_pos, siguiente_indice, visitados):
                        return True
        
        return False
    
    for pos_inicial in indices_por_palabra[0]:
        if buscar_secuencia(pos_inicial, 0):
            return True
            
    return False

# Función para verificar si un comentario debe excluirse de una categoría
def debe_excluirse(comentario, categoria, dic_exclusiones):
    if categoria not in dic_exclusiones:
        return False
    
    for patron_exclusion in dic_exclusiones[categoria]:
        if buscar_patron_avanzado(comentario, patron_exclusion, max_gap=6):
            return True
    
    return False

# Función modificada para tener en cuenta las exclusiones
def obtener_categorias_n1_mejorado(comentario_limpio, dic_n1, dic_exclusiones):
    categorias_encontradas = []
    puntuaciones_categorias = {}
    
    for categoria_n1, tipos in dic_n1.items():
        # Primero verificar si el comentario debe excluirse de esta categoría
        if debe_excluirse(comentario_limpio, categoria_n1, dic_exclusiones):
            continue
            
        for frase in tipos["exacto"]:
            if buscar_patron_exacto(comentario_limpio, frase):
                puntuaciones_categorias[categoria_n1] = 100
                categorias_encontradas.append(categoria_n1)
                break
    
    if categorias_encontradas:
        return categorias_encontradas
    
    for categoria_n1, tipos in dic_n1.items():
        # Verificar exclusiones antes de buscar patrones
        if debe_excluirse(comentario_limpio, categoria_n1, dic_exclusiones):
            continue
            
        for frase in tipos["patron"]:
            if buscar_patron_avanzado(comentario_limpio, frase, max_gap=6):
                palabras = frase.split()
                puntuacion = len(palabras) * 5
                
                palabras_significativas = [p for p in palabras if not es_palabra_comun(p)]
                puntuacion += len(palabras_significativas) * 10
                
                if categoria_n1 not in puntuaciones_categorias or puntuacion > puntuaciones_categorias[categoria_n1]:
                    puntuaciones_categorias[categoria_n1] = puntuacion
    
    categorias_ordenadas = sorted(puntuaciones_categorias.keys(), 
                                 key=lambda c: puntuaciones_categorias[c],
                                 reverse=True)
    
    categorias_filtradas = [c for c in categorias_ordenadas 
                           if puntuaciones_categorias[c] >= 15]
    
    return categorias_filtradas[:1] if categorias_filtradas else []

def obtener_subcategorias_n2_mejorado(comentario_limpio, categoria_n1, dic_n2):
    subcategorias_encontradas = []
    puntuaciones_subcategorias = {}
    
    if categoria_n1 in dic_n2:
        for subcategoria_n2, tipos in dic_n2[categoria_n1].items():
            for frase in tipos["exacto"]:
                if buscar_patron_exacto(comentario_limpio, frase):
                    puntuaciones_subcategorias[subcategoria_n2] = 100
                    subcategorias_encontradas.append(subcategoria_n2)
                    break
        
        if subcategorias_encontradas:
            return subcategorias_encontradas
        
        for subcategoria_n2, tipos in dic_n2[categoria_n1].items():
            for frase in tipos["patron"]:
                if buscar_patron_avanzado(comentario_limpio, frase, max_gap=6):
                    palabras = frase.split()
                    puntuacion = len(palabras) * 5
                    
                    palabras_significativas = [p for p in palabras if not es_palabra_comun(p)]
                    puntuacion += len(palabras_significativas) * 10
                    
                    if subcategoria_n2 not in puntuaciones_subcategorias or puntuacion > puntuaciones_subcategorias[subcategoria_n2]:
                        puntuaciones_subcategorias[subcategoria_n2] = puntuacion
        
        subcategorias_ordenadas = sorted(puntuaciones_subcategorias.keys(), 
                                        key=lambda c: puntuaciones_subcategorias[c],
                                        reverse=True)
        
        subcategorias_filtradas = [c for c in subcategorias_ordenadas 
                                  if puntuaciones_subcategorias[c] >= 15]
        
        return subcategorias_filtradas[:1] if subcategorias_filtradas else []
    
    return []

def obtener_subcategorias_n3_mejorado(comentario_limpio, categoria_n1, subcategoria_n2, dic_n3):
    subcategorias_encontradas = []
    puntuaciones_subcategorias = {}
    
    if categoria_n1 in dic_n3 and subcategoria_n2 in dic_n3[categoria_n1]:
        for subcategoria_n3, tipos in dic_n3[categoria_n1][subcategoria_n2].items():
            for frase in tipos["exacto"]:
                if buscar_patron_exacto(comentario_limpio, frase):
                    puntuaciones_subcategorias[subcategoria_n3] = 100
                    subcategorias_encontradas.append(subcategoria_n3)
                    break
        
        if subcategorias_encontradas:
            return subcategorias_encontradas
        
        for subcategoria_n3, tipos in dic_n3[categoria_n1][subcategoria_n2].items():
            for frase in tipos["patron"]:
                if buscar_patron_avanzado(comentario_limpio, frase, max_gap=6):
                    palabras = frase.split()
                    puntuacion = len(palabras) * 5
                    
                    palabras_significativas = [p for p in palabras if not es_palabra_comun(p)]
                    puntuacion += len(palabras_significativas) * 10
                    
                    if subcategoria_n3 not in puntuaciones_subcategorias or puntuacion > puntuaciones_subcategorias[subcategoria_n3]:
                        puntuaciones_subcategorias[subcategoria_n3] = puntuacion
        
        subcategorias_ordenadas = sorted(puntuaciones_subcategorias.keys(), 
                                        key=lambda c: puntuaciones_subcategorias[c],
                                        reverse=True)
        
        subcategorias_filtradas = [c for c in subcategorias_ordenadas 
                                  if puntuaciones_subcategorias[c] >= 15]
        
        return subcategorias_filtradas[:1] if subcategorias_filtradas else []
    
    return []

def analizar_sentimiento_mejorado(comentario):
    palabras_positivas = {
        "bien": 1, "buena": 1, "bueno": 1, "excelente": 2, "ok": 0.5, "perfecto": 2, 
        "correcto": 1, "satisfecho": 2, "agradable": 1, "rapido": 1, "eficiente": 1, 
        "util": 1, "practico": 1, "facil": 1, "sencillo": 1, "mejor": 1, "genial": 2,
        "encanta": 2, "gusta": 1, "contento": 1, "gracias": 1, "felicitaciones": 2,
        "adecuado": 0.5, "ideal": 1, "feliz": 1, "maravilloso": 2, "increible": 2
    }
    
    palabras_negativas = {
        "mal": 1, "mala": 1, "malo": 1, "pesimo": 2, "terrible": 2, "horrible": 2, 
        "lento": 1, "deficiente": 1, "problema": 1, "error": 1, "falla": 1, 
        "no sirve": 2, "no funciona": 2, "complicado": 1, "dificil": 1, "peor": 1.5,
        "molesto": 1, "molestia": 1, "inutil": 1.5, "fatal": 2, "desastre": 2,
        "frustrante": 1.5, "cansado": 0.5, "demorado": 1, "engorroso": 1, "confuso": 1,
        "aburrido": 0.5, "tedioso": 1, "queja": 1, "reclamo": 1.5, "decepciona": 1.5
    }
    
    negaciones = ["no", "ni", "nunca", "jamas", "tampoco", "nada", "ningun", "ninguna"]
    
    intensificadores = ["muy", "mucho", "demasiado", "bastante", "extremadamente", "super", "totalmente"]
    
    palabras = comentario.lower().split()
    sentimiento = 0
    negacion_activa = False
    intensificador_activo = False
    
    for i, palabra in enumerate(palabras):
        if palabra in negaciones:
            negacion_activa = True
            continue
            
        if palabra in intensificadores:
            intensificador_activo = True
            continue
            
        palabra_encontrada = False
        for p_pos, valor in palabras_positivas.items():
            if ' ' in p_pos:
                if i + len(p_pos.split()) <= len(palabras):
                    frase_comentario = ' '.join(palabras[i:i+len(p_pos.split())])
                    if frase_comentario == p_pos:
                        palabra_encontrada = True
                        if negacion_activa:
                            sentimiento -= valor * 1.2
                        else:
                            sentimiento += valor * (1.5 if intensificador_activo else 1)
                        break
            elif similitud_palabras_mejorada(palabra, p_pos) > 0.9:
                palabra_encontrada = True
                if negacion_activa:
                    sentimiento -= valor * 1.2
                else:
                    sentimiento += valor * (1.5 if intensificador_activo else 1)
                break
        
        if not palabra_encontrada:
            for p_neg, valor in palabras_negativas.items():
                if ' ' in p_neg:
                    if i + len(p_neg.split()) <= len(palabras):
                        frase_comentario = ' '.join(palabras[i:i+len(p_neg.split())])
                        if frase_comentario == p_neg:
                            if negacion_activa:
                                sentimiento += valor * 0.8
                            else:
                                sentimiento -= valor * (1.5 if intensificador_activo else 1)
                            break
                elif similitud_palabras_mejorada(palabra, p_neg) > 0.9:
                    if negacion_activa:
                        sentimiento += valor * 0.8
                    else:
                        sentimiento -= valor * (1.5 if intensificador_activo else 1)
                    break
        
        negacion_activa = False
        intensificador_activo = False
    
    if len(palabras) <= 5 and sentimiento != 0:
        sentimiento *= 1.2
        
    if sentimiento > 1:
        return "positivo"
    elif sentimiento < -0.5:
        return "negativo"
    else:
        return "neutral"

def es_comentario_generico(comentario):
    palabras_genericas = ["todo", "nada", "algo", "mucho", "poco", "bien", "mal"]
    palabras = comentario.split()
    
    if len(palabras) <= 2:
        for palabra in palabras:
            if palabra in palabras_genericas:
                return True
    
    palabras_significativas = [p for p in palabras if not es_palabra_comun(p)]
    
    if len(palabras_significativas) < max(1, len(palabras) * 0.3):
        return True
    
    return False

try:
    df = spark.sql(f"""
        SELECT 
            ID_ENCUESTA, 
            CODINTERNOCOMPUTACIONAL, 
            CANAL, 
            '' AS EDAD, 
            FECHA_tRANSACCION AS FEC_TRANSACCION, 
            FECHA_RESPUESTA as FEC_RESPUESTA, 
            SEGMENTO, 
            NPS AS P1_NPS, 
            COMENTARIO_NPS
        FROM catalog_lhcl_prod_bcp.bcp_edv_expclie_001.banca_online_resp 
        WHERE codmes in ( '202412', '202501', '202502', '202503') 
        and canal = 'Banca Móvil' 
        and NPS in ('0', '1', '2', '3', '4', '5', '6')
    """)

    df = df.toPandas()
    print(f"Datos cargados: {len(df)} registros")
except Exception as e:
    print(f"Error al ejecutar la consulta SQL: {str(e)}")
    print("Continuando con DataFrame vacío...")
    df = pd.DataFrame(columns=[
        "ID_ENCUESTA", "CODINTERNOCOMPUTACIONAL", "CANAL", "EDAD",
        "FEC_TRANSACCION", "FEC_RESPUESTA", "SEGMENTO", "P1_NPS", "COMENTARIO_NPS"
    ])

n1_path = "/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n1.txt.txt"
n2_path = "/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n2.txt.txt"
n3_path = "/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n3.txt.txt"
excludes_path = "/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/excludes.txt"

try:
    print(f"Cargando archivo N1: {n1_path}")
    dic_n1 = cargar_n1(n1_path)
    print(f"Archivo N1 cargado: {len(dic_n1)} categorías")
    
    print(f"Cargando archivo N2: {n2_path}")
    dic_n2 = cargar_n2(n2_path, dic_n1)
    print(f"Archivo N2 cargado: categorías para {len(dic_n2)} categorías N1")
    
    print(f"Cargando archivo N3: {n3_path}")
    dic_n3 = cargar_n3(n3_path, dic_n1, dic_n2)
    print(f"Archivo N3 cargado")
    
    # Cargar exclusiones
    print(f"Cargando archivo de exclusiones: {excludes_path}")
    dic_exclusiones = cargar_exclusiones(excludes_path)
    print(f"Archivo de exclusiones cargado: {len(dic_exclusiones)} categorías con exclusiones")
    
    total_patrones_n1 = sum(len(cat["patron"]) + len(cat["exacto"]) for cat in dic_n1.values())
    print(f"Total de patrones N1: {total_patrones_n1}")
    
    total_patrones_n2 = 0
    for cat_n1, subcats in dic_n2.items():
        for subcat, tipos in subcats.items():
            total_patrones_n2 += len(tipos["patron"]) + len(tipos["exacto"])
    print(f"Total de patrones N2: {total_patrones_n2}")
    
    total_patrones_n3 = 0
    for cat_n1, subcats_n2 in dic_n3.items():
        for subcat_n2, subcats_n3 in subcats_n2.items():
            for subcat_n3, tipos in subcats_n3.items():
                total_patrones_n3 += len(tipos["patron"]) + len(tipos["exacto"])
    print(f"Total de patrones N3: {total_patrones_n3}")
    
    # Mostrar resumen de exclusiones
    total_exclusiones = sum(len(exclusiones) for exclusiones in dic_exclusiones.values())
    print(f"Total de patrones de exclusión: {total_exclusiones}")
    for categoria, exclusiones in dic_exclusiones.items():
        print(f"  - {categoria}: {len(exclusiones)} exclusiones")
    
except Exception as e:
    print(f"Error al cargar los archivos de patrones: {str(e)}")
    sys.exit(1)

print("Iniciando procesamiento de comentarios...")
inicio = datetime.now()
filas_resultantes = []
total = len(df)

for idx, row in df.iterrows():
    if idx % 100 == 0 and idx > 0:
        print(f"Procesados {idx}/{total} comentarios ({idx/total*100:.1f}%)...")
        
    comentario_original = row["COMENTARIO_NPS"] if not pd.isna(row["COMENTARIO_NPS"]) else ""
    comentario_limpio = limpiar_texto(comentario_original)
    
    fecha_transaccion = row["FEC_TRANSACCION"]
    codmes = ""
    if not pd.isna(fecha_transaccion):
        try:
            if isinstance(fecha_transaccion, str):
                fecha_dt = pd.to_datetime(fecha_transaccion)
            else:
                fecha_dt = fecha_transaccion
                
            codmes = fecha_dt.strftime("%Y%m")
        except:
            codmes = ""
    
    if comentario_limpio == "":
        cats_n1 = ["Sin Comentario"]
        cats_n2 = ["Sin Comentario"]
        cats_n3 = ["Sin Comentario"]
        sentimiento = "neutral"
    else:
        sentimiento = analizar_sentimiento_mejorado(comentario_limpio)
        
        es_generico = es_comentario_generico(comentario_limpio)
        num_palabras = len(comentario_limpio.split())
        
        if num_palabras <= 7 and num_palabras > 0:  
            cats_n1 = obtener_categorias_n1_mejorado(comentario_limpio, dic_n1, dic_exclusiones)
            if not cats_n1:  
                cats_n1 = ["noespecifica"]
        else:
            cats_n1 = obtener_categorias_n1_mejorado(comentario_limpio, dic_n1, dic_exclusiones)
            
            if not cats_n1:
                cats_n1 = ["Sin categoría N1"]
    
    for c1 in cats_n1:
        if c1 == "Sin Comentario" or c1 == "Sin categoría N1" or c1 == "noespecifica":
            cats_n2 = ["Sin categoría N2"]
            cats_n3 = ["Sin categoría N3"]
        else:
            cats_n2 = obtener_subcategorias_n2_mejorado(comentario_limpio, c1, dic_n2)
            
            if not cats_n2:
                cats_n2 = ["Sin categoría N2"]
                cats_n3 = ["Sin categoría N3"]
            else:
                temp_cats_n3 = []
                for c2 in cats_n2:
                    if c2 == "Sin categoría N2":
                        temp_cats_n3.append("Sin categoría N3")
                    else:
                        c3_list = obtener_subcategorias_n3_mejorado(comentario_limpio, c1, c2, dic_n3)
                        if c3_list:
                            temp_cats_n3.extend(c3_list)
                        else:
                            temp_cats_n3.append("Sin categoría N3")
                cats_n3 = temp_cats_n3
        
        for i, c2 in enumerate(cats_n2):
            c3 = cats_n3[i] if i < len(cats_n3) else "Sin categoría N3"
            
            fila_resultado = {col: row[col] for col in row.index if col in row}
            
            fila_resultado.update({
                "CODMES": codmes,
                # "NPS_Comentario": comentario_original,
                "Comentario_Limpio": comentario_limpio,
                "Categoria_N1": c1,
                "Categoria_N2": c2,
                "Categoria_N3": c3,
                "Sentimiento": sentimiento,
                "Palabras": len(comentario_limpio.split()) if comentario_limpio else 0,
                "Usa_Exclusiones": "Sí"  # Nueva columna para indicar que se utilizó la lógica de exclusiones
            })
            
            filas_resultantes.append(fila_resultado)

df_categorizado = pd.DataFrame(filas_resultantes)

fin = datetime.now()
tiempo_ejecucion = (fin - inicio).total_seconds()

print("\n===== RESULTADOS DE LA CLASIFICACIÓN =====")
print(f"Total registros procesados: {total}")
print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos ({tiempo_ejecucion/60:.2f} minutos)")

print("\nDistribución por categoría N1:")
for cat, count in df_categorizado["Categoria_N1"].value_counts().items():
    print(f"  - {cat}: {count} ({count/len(df_categorizado)*100:.1f}%)")

print("\nDistribución por categoría N2:")
for cat, count in df_categorizado["Categoria_N2"].value_counts().head(10).items():
    print(f"  - {cat}: {count} ({count/len(df_categorizado)*100:.1f}%)")

print("\nDistribución por categoría N3:")
for cat, count in df_categorizado["Categoria_N3"].value_counts().head(10).items():
    print(f"  - {cat}: {count} ({count/len(df_categorizado)*100:.1f}%)")

print("\nDistribución de sentimiento:")
for sent, count in df_categorizado["Sentimiento"].value_counts().items():
    print(f"  - {sent}: {count} ({count/len(df_categorizado)*100:.1f}%)")

excel_output = "/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/EXPORT_topicos_digitales_nuevaversion.xlsx"

try:
    print(f"\nGuardando resultados en Excel: {excel_output}")
    df_categorizado.to_excel(excel_output, index=False, encoding="Windows-1252")
    print("Archivo Excel guardado correctamente.")
except Exception as e:
    print(f"Error al guardar en Excel: {str(e)}")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_path = f"/Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/EXPORT_topicos_digitales_{timestamp}.csv"
    try:
        print(f"Intentando guardar como CSV en: {backup_path}")
        df_categorizado.to_csv(backup_path, index=False, encoding="utf-8")
        print("Archivo CSV de respaldo guardado correctamente.")
    except Exception as e2:
        print(f"Error al guardar archivo de respaldo: {str(e2)}")

print("\nProceso completado.")

Datos cargados: 8365 registros
Cargando archivo N1: /Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n1.txt.txt
Archivo N1 cargado: 5 categorías
Cargando archivo N2: /Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n2.txt.txt
Archivo N2 cargado: categorías para 5 categorías N1
Cargando archivo N3: /Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/n3.txt.txt
Archivo N3 cargado
Cargando archivo de exclusiones: /Workspace/Users/jenisalvarado@bcp.com.pe/GENERACION BASES | CARGA DE RESPUESTAS | TIPIFICACION COMENTARIOS/Herramienta_comentarios/excludes.txt
Archivo de exclusiones cargado: 4 categorías con exclusiones
Total de patrones N1: 1545
Total de patrones N2: 2566
Total de patrones N3: 2042
Total de patrones de exclusión: 340
  - funcionalidad